In [19]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa

import matplotlib.pyplot as plt

In [16]:
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error

def bias(x, y): return np.mean(x - y)

def get_stats(x, y):
    if type(x) == xr.DataArray: x = x.values.ravel()
    if type(y) == xr.DataArray: y = y.values.ravel()
    idx = (~np.isnan(x)) & (~np.isnan(y))
    x, y = x[idx], y[idx]
    r, p = pearsonr(x, y)
    b = bias(x, y)
    mae = mean_absolute_error(x, y)
    rmse = mean_squared_error(x, y, squared = False)
    n = len(x)
    return r, b, mae, rmse, n

In [7]:
data_dir = Path('~/spicy-snow/SnowEx-Data').expanduser()
dss = {fp.stem:xr.open_dataset(fp) for fp in list(data_dir.glob('*.nc'))}

In [24]:
for name, ds in dss.items():

    if name == 'Frasier_2020-02-11':
        im_date = pd.to_datetime('2020-02-16')
    else:
        im_date = pd.to_datetime(ds.sel(time = ds.attrs['lidar-flight-time'], method = 'nearest').time.values.ravel()[0])

    r, b, mae, rmse, n  = get_stats(ds['lidar-sd'], ds['snow_depth'].sel(time = im_date, method = 'nearest'))
    print(r, rmse, n)

    ds_500 = ds.coarsen(x = 6, y = 6, boundary = 'pad').mean()
    r, b, mae, rmse,n  = get_stats(ds_500['lidar-sd'], ds_500['snow_depth'].sel(time = im_date, method = 'nearest'))
    print(r, rmse, n)
    print('-----')

0.07852781182727887 1.1094484136851095 3508
0.34593399838825206 0.8293901748624931 129
-----
0.15540926973408717 0.7093077489076663 6604
0.2669001352481223 0.5745167636325517 223
-----
0.16574099592153635 0.7859482832935086 3567
0.28727489459880196 0.5926015810472951 138
-----
0.330321586526359 0.9414765634178163 16317
0.5454760933001146 0.5287613140607289 562
-----
0.3209910445417788 1.2811694504771576 2646
0.5717240589133219 0.7263829572612499 114
-----
0.32504615751428695 0.9636948982667913 3541
0.5797942008118455 0.6417445020023004 132
-----
0.3077100439782632 1.0381160082187428 16107
0.6019496351886001 0.7874109322867167 546
-----
0.2596788884317362 0.968756660602673 3701
0.47984214953200977 0.8311807631767741 128
-----
0.0686484157773657 1.1559963757264684 2200
0.1714374867867996 0.926968115168757 84
-----
